In [2]:
import cv2
import numpy as np

# 加载图像
image1 = cv2.imread('./images/1.jpg')
image2 = cv2.imread('./images/2.jpg')
image3 = cv2.imread('./images/3.jpg')

# 检查所有图像的尺寸是否相同
height1, width1 = image1.shape[:2]
height2, width2 = image2.shape[:2]
height3, width3 = image3.shape[:2]

# 假设所有图像的高度和宽度相同，简化拼接过程
if height1 == height2 == height3 and width1 == width2 == width3:
    # 水平方向上拼接图像
    concatenated_image = np.concatenate((image1, image2, image3), axis=1)
    # 保存拼接后的图像
    cv2.imwrite('concatenated_image.jpg', concatenated_image)
    cv2.imshow('Concatenated Image', concatenated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Images have different dimensions and cannot be concatenated directly.")


In [4]:
import cv2
import numpy as np

def stitch_images(img1, img2):
    # 初始化ORB检测器
    orb = cv2.ORB_create(5000)

    # 寻找关键点和描述符
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # 创建匹配器
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # 匹配描述符
    matches = matcher.match(des1, des2)

    # 根据距离排序
    matches = sorted(matches, key=lambda x: x.distance)

    # 只取最好的一些匹配点
    good_matches = matches[:int(len(matches) * 0.15)]

    # 从好的匹配中提取位置
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    # 用RANSAC算法计算仿射变换
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # 使用变换矩阵将图像img1变换到img2的坐标系
    h, w = img2.shape[:2]
    result = cv2.warpPerspective(img1, M, (w, h))

    # 把img2加到变换后的img1上
    result[0:h, 0:w] = img2

    return result

# 加载图像
image1 = cv2.imread('./images/1.jpg')
image2 = cv2.imread('./images/2.jpg')
image3 = cv2.imread('./images/3.jpg')

# 首先拼接图像1和图像2
result12 = stitch_images(image1, image2)

# 然后拼接结果和图像3
final_result = stitch_images(result12, image3)

# 显示和保存最终结果
cv2.imshow('Final Stitched Image', final_result)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('final_stitched_result.jpg', final_result)


True